<a href="https://colab.research.google.com/github/Next-Sunshine/TTF0318/blob/master/TFFTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#test environment
!pip install --quiet --upgrade tensorflow_federated

import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

tf.compat.v1.enable_v2_behavior()
@tff.federated_computation
def hello_world():
  return 'Hello World!'

hello_world()

b'Hello World!'

In [0]:
#定义一组联邦浮点类型的值，分布式传感器阵列中出现的温度读数的集合就可以定义成这种
#put tf.float32 into the placement tff.CLIENTS
#定义一组联邦值，member是float32，placement是CLIENTS
federated_float_on_clients = tff.FederatedType(tf.float32,tff.CLIENTS)   
#the function str() turn int to a string
#为什么多个str()不会依次运行只会运行最后一个,我猜测可能是用一个缓冲区装串，每写一次就进行覆盖
#对于一个一维线性模型，构造float32类型的参数ab的一对非联邦类型以供TFF使用

In [0]:
#查看federated_float_on_clients的成员类型？
str(federated_float_on_clients.member)

'float32'

In [0]:
#federated_float_on_clients reperents a relationship whose member shows the member constituents and placement shows the place we putted the member in
str(federated_float_on_clients.placement)  

'CLIENTS'

In [0]:
#placement G with member constituents T can be repersented as {T}@G,here output {float32}@CLIENTS
#查看联邦类型federated_float_on_clients，会打印出它的member以及placement
str(federated_float_on_clients)   

'{float32}@CLIENTS'

In [0]:
#测试FederatedType构造函数中all_equal的默认值是false，默认并不是相同的类型
federated_float_on_clients.all_equal  

False

In [0]:
#通过控制all_equal=True来显式说明所有的member都是相同的类型，那么这时{}就会去掉
str(tff.FederatedType(tf.float32,tff.CLIENTS,all_equal=True))

'float32@CLIENTS'

In [0]:
#一对float32类型的参数，构造一个名叫simple_regression_model_type作为member类型的数以便在tff中使用
#相当于federated_float_on_clients = tff.FederatedType(tf.float32,tff.CLIENTS)中的tf.float
simple_regression_model_type = (
    tff.NamedTupleType([('a',tf.float32),('b',tf.float32)])
)
str(simple_regression_model_type)

'<a=float32,b=float32>'

In [0]:
#当模型广播给客户机时，得到的联邦值类型可以表示为如下
str(tff.FederatedType(simple_regression_model_type,tff.CLIENTS,all_equal=True))

'<a=float32,b=float32>@CLIENTS'

In [0]:
#Federated computation example：计算传感器阵列报告的平均温度
@tff.federated_computation(tff.FederatedType(tf.float32,tff.CLIENTS))
def get_average_temperature(sensor_readings):

  #添加打印以监听添加了@tff.federated_computation的函数,另外猜测"{}"是占位符，然后用format填充
  print('Getting traced, the argument is "{}".'
  .format(type(sensor_readings).__name__))

  return tff.federated_mean(sensor_readings)
#使用type_signature打印计算的类型,({float32}@CLIENTS -> float32@SERVER)
#类型签名告诉我们,计算接受客户端设备上不同传感器读数的集合，并在服务器上返回单个平均值。
str(get_average_temperature.type_signature)

#使用纯列表的方式,32位浮点数类型
get_average_temperature([68.5,70.3,69.8])

Getting traced, the argument is "ValueImpl".


'(float32 -> float32)'

In [4]:
#定义TensorFlow计算，接受一个数字并加0.5，@tff.tf_computation()中放的是参数类型吗，此处是一个tf类型的浮点数，所以用@tff.tf_修饰
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x,0.5)
#打印类型签名
#(float32 -> float32):类型签名告诉我们，函数接收一个浮点数并且返回一个浮点数
str(add_half.type_signature)

'(float32 -> float32)'

In [0]:
#此处tff.federated_computaion（）中放的是一个联邦类型的值，member是float32，placement是CLIENTS
@tff.federated_computation(tff.FederatedType(tf.float32,tff.CLIENTS))
def add_half_on_clients(x):
  return tff.federated_map(add_half,x)
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

In [0]:
#此处输出的结果好像是在显示张量
add_half_on_clients([1.0,3.0,2.0])

[<tf.Tensor: shape=(), dtype=float32, numpy=1.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=3.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.5>]

In [0]:
try:
  # Eager mode,每次重新进入运行新的代码都要先运行一下环境
  #运行失败的原因是在add_ten()之外又构造了一个常量
  #tf.constant(.10)构造浮点常量10，如果是（10）则是构造整型常量10
  constant_10 = tf.constant(10.)#这一行如果去掉把里面的constant_10改成tf.constant(10.)就不会报错

  @tff.tf_computation(tf.float32)
  def add_ten(x):
    return x + constant_10

except Exception as err:
  print(err)

In [0]:
def get_constant_10():
  return tf.constant(10.)
@tff.tf_computation(tf.float32)
def add_ten(x):
  return x + get_constant_10()

add_ten(5.0)

15.0

In [0]:
float32_sequence = tff.SequenceType(tf.float32)
str(float32_sequence)

'float32*'

In [0]:
@tff.tf_computation(tff.SequenceType(tf.float32))
def get_local_temperature_average(local_temperatures):
  sum_and_count = (
      #Python中的reduce方法会用传给它的函数先对1,2个参数进行操作然后再用得到的结果和第三个元素操作
      #这里使用lambda匿名函数，x[0]+y应该是在计算和，x[1]+1应该是在计算个数
      local_temperatures.reduce((0.0,0),lambda x, y: (x[0] + y, x[1] + 1))
  )
  return sum_and_count[0] / tf.cast(sum_and_count[1],tf.float32)
str(get_local_temperature_average.type_signature)

'(float32* -> float32)'

In [0]:
@tff.tf_computation(tff.SequenceType(tf.int32))
def foo(x):
  return x.reduce(np.int32(0), lambda x, y: x+y)
foo([1,2,3])

6

In [0]:
get_local_temperature_average([68.5,70.3,69.8])

69.53333

In [0]:
@tff.tf_computation(tff.SequenceType(collections.OrderedDict([('A', tf.int32),('B', tf.int32)])))
def foo(ds):
  print('element_structure = {}'.format(ds.element_spec))
  return ds.reduce(np.int32(0), lambda total, x: total + x['A'] * x['B'])

element_structure = OrderedDict([('A', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('B', TensorSpec(shape=(), dtype=tf.int32, name=None))])


In [0]:
str(foo.type_signature)

'(<A=int32,B=int32>* -> int32)'

In [0]:
foo([{'A': 2, 'B':3}, {'A':4, 'B':5}])

26

In [0]:
@tff.federated_computation(
    tff.FederatedType(tff.SequenceType(tf.float32), tff.CLIENTS)
)
def get_global_temperature_average(sensor_readings):
  return tff.federated_mean(
      tff.federated_map(get_local_temperature_average, sensor_readings)
  )

In [0]:
str(get_global_temperature_average.type_signature)

'({float32*}@CLIENTS -> float32@SERVER)'